In [1]:
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
import nrrd
import numpy as np
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from subtle.subtle_preprocess import scale_im as hist_match, register_im

import matplotlib.pyplot as plt
plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (12, 10)

import SimpleITK as sitk

def combine_seg_classes(fpath_seg):
    seg_sitk = sitk.ReadImage(fpath_seg)
    seg = (sitk.GetArrayFromImage(seg_sitk) > 0).astype(np.uint8)
    seg_sitk_new = sitk.GetImageFromArray(seg)
    seg_sitk_new.CopyInformation(seg_sitk)
    return seg_sitk_new

def get_feature_values(df_radiomics, cls, feature):
    df_filt = df_radiomics.query(f'Class == "{cls}" and Feature == "{feature}"')
    
    dict_vals = df_filt.to_dict(orient='records')
    gt_vals = np.array([row['Value'] for row in dict_vals if row['Image'] == 'GT'])
    syn_vals = np.array([row['Value'] for row in dict_vals if row['Image'] == 'Syn'])
    
    return gt_vals, syn_vals

def ccc(x, y):
    ''' Concordance Correlation Coefficient'''
    sxy = np.sum((x - x.mean())*(y - y.mean()))/x.shape[0]
    rhoc = 2*sxy / (np.var(x) + np.var(y) + (x.mean() - y.mean())**2)
    return rhoc

def pcc(x, y):
    ''' Pearson Correlation Coefficient'''
    sxy = np.sum((x - x.mean())*(y - y.mean()))/x.shape[0]
    rho = sxy / (np.std(x)*np.std(y))
    return rho

def dr(x, y):
    max_k = np.max([np.max(x), np.max(y)])
    min_k = np.min([np.min(x), np.min(y)])
    
    smn = np.mean([np.abs(x[i] - y[i]) / (max_k - min_k) for i in np.arange(x.shape[0])])
    
    return 1 - smn

def norm_pixel_values(img_sitk, ref_sitk, register=False):
    img = sitk.GetArrayFromImage(img_sitk).astype(np.float32)
    img_ref = sitk.GetArrayFromImage(ref_sitk).astype(np.float32)
    
    img = hist_match(img_ref, img)
    
    if register:
        pmap = sitk.GetDefaultParameterMap('affine')
        img_sitk_new, _ = register_im(
            img_ref, img, ref_fixed=ref_sitk, ref_moving=img_sitk, return_sitk_img=True, param_map=pmap
        )
        return img_sitk_new
    
    img_sitk_new = sitk.GetImageFromArray(img)
    img_sitk_new.CopyInformation(img_sitk)
    return img_sitk_new

/home/srivathsa/miniconda3/envs/mmt/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/srivathsa/projects/SubtleGad/train/subtle/utils/hyperparameter.py:8: UserWarning: Module test_tube not found - hyperparameter related functions cannot be used
  warnings.warn('Module test_tube not found - hyperparameter related functions cannot be used')


<Figure size 432x288 with 0 Axes>

In [5]:
extractor = featureextractor.RadiomicsFeatureExtractor('Params-generic.yaml')
ftr_classes = ['shape', 'firstorder', 'glcm', 'glrlm', 'glszm', 'gldm', 'ngtdm']

In [ ]:
dirpath_cases = '/home/srivathsa/projects/studies/gad/mmt_seg/seg/cases'
dirpath_seg = '/home/srivathsa/projects/studies/gad/mmt_seg/seg/pred_mmt'

ftr_list = []

cases = sorted([d.split('/')[-1] for d in glob('{}/*'.format(dirpath_cases)) if 'syn' not in d])
for cnum in tqdm(cases, total=len(cases)):
    img = sitk.ReadImage(f'{dirpath_cases}/{cnum}/{cnum}_T1Gd.nii.gz')
    img_syn = norm_pixel_values(
        sitk.ReadImage(f'{dirpath_cases}/{cnum}_T1Gd_syn/{cnum}_T1Gd_syn_T1Gd.nii.gz'), img, register=True
    )
    seg = combine_seg_classes(f'{dirpath_seg}/{cnum}/{cnum}_seg.nii.gz')
    seg_syn = combine_seg_classes(f'{dirpath_seg}/{cnum}_T1Gd_syn/{cnum}_T1Gd_syn_seg.nii.gz')
    
    ftr_gt = extractor.execute(img, seg)
    ftr_syn = extractor.execute(img_syn, seg)
    
    for ftr_str in ftr_gt.keys():
        kw, cls, ftr_name = ftr_str.split('_')
        if cls not in ftr_classes: continue
            
        row_gt = {'Case': cnum, 'Image': 'GT'}
        row_syn = {'Case': cnum, 'Image': 'Syn'}
        
        fv_gt = ftr_gt[ftr_str]
        fv_syn = ftr_syn[ftr_str]
        
        row_gt['Class'] = cls
        row_syn['Class'] = cls
                
        row_gt['Feature'] = ftr_name
        row_syn['Feature'] = ftr_name
        row_gt['Value'] = fv_gt
        row_syn['Value'] = fv_syn
        
        ftr_list.append(row_gt)
        ftr_list.append(row_syn)

In [6]:
df_radiomics = pd.DataFrame(ftr_list)
df_radiomics.to_csv('/home/srivathsa/projects/studies/gad/mmt_seg/radiomics.csv')

In [31]:
len(set(df_radiomics['Feature']))

99

In [25]:
csv_fname = 'radiomics'
df_radiomics = pd.read_csv(f'/home/srivathsa/projects/studies/gad/mmt_seg/{csv_fname}.csv', index_col=0)
ftr_cls_list = []

for ftr_cls in ftr_classes:
    ftr_names = list(set(df_radiomics.query(f'Class == "{ftr_cls}"')['Feature']))
    ftr_cls_list.extend([(ftr_cls, fn) for fn in ftr_names])
    
corr_list = []
for (cls, ftr) in ftr_cls_list:
    x, y = get_feature_values(df_radiomics, cls=cls, feature=ftr)
    corr_list.append({'Class': cls, 'Feature': ftr, 'CCC': ccc(x, y)})

In [27]:
df_corr = pd.DataFrame(corr_list)
df_corr = df_corr.query('CCC > 0.7')
df_corr.groupby("Class").aggregate(['mean', 'std'])

CCC          
                mean       std
Class                         
firstorder  0.803166  0.080497
glcm        0.733335  0.023584
gldm        0.846263  0.051079
glrlm       0.786603  0.066473
glszm       0.806873  0.059276
ngtdm       0.843695  0.112970
shape       1.000000  0.000000

In [28]:
df_corr.query('Class == "shape"')

,Class,Feature,CCC
0,shape,Maximum2DDiameterColumn,1.0
1,shape,Maximum2DDiameterSlice,1.0
2,shape,Elongation,1.0
3,shape,SurfaceVolumeRatio,1.0
4,shape,MinorAxisLength,1.0
5,shape,LeastAxisLength,1.0
6,shape,MeshVolume,1.0
7,shape,VoxelVolume,1.0
8,shape,Maximum2DDiameterRow,1.0
9,shape,SurfaceArea,1.0


In [ ]:
df_radiomics.query('Class == "glcm" and Feature == "Imc1"')

In [ ]:
img_arr = sitk.GetArrayFromImage(img)
img_syn_arr = sitk.GetArrayFromImage(img_syn)

plt.imshow(np.hstack([img_arr[40], img_syn_arr[40]]))

In [ ]:
print(img_arr.min(), img_arr.max())

In [ ]:
print(img_syn_arr.min(), img_syn_arr.max())

In [ ]:
from radiomics.glcm import RadiomicsGLCM
bw = 1
glcm = RadiomicsGLCM(img, seg, binWidth=bw)

glcm2 = RadiomicsGLCM(img_syn, seg, binWidth=bw)

In [ ]:
plt.imshow(np.hstack([glcm.imageArray[40], glcm2.imageArray[40]]))

print(np.unique(glcm.imageArray))

In [ ]:
img_arr = sitk.GetArrayFromImage(img)
print(img_arr.min(), img_arr.max())

In [ ]:
print(glcm.getClusterTendencyFeatureValue())
print(glcm2.getClusterTendencyFeatureValue())